In [0]:
%sh
curl https://packages.microsoft.com/keys/microsoft.asc | apt-key add -
curl https://packages.microsoft.com/config/ubuntu/16.04/prod.list > /etc/apt/sources.list.d/mssql-release.list
sudo apt-get update
sudo ACCEPT_EULA=Y apt-get -q -y install msodbcsql17

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   983  100   983    0     0  19660      0 --:--:-- --:--:-- --:--:-- 19660
OK
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    79  100    79    0     0   1837      0 --:--:-- --:--:-- --:--:--  1880
Hit:1 https://packages.microsoft.com/ubuntu/16.04/prod xenial InRelease
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:3 https://repos.azul.com/zulu/deb stable InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Get:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:6 http://security.ubuntu.com/ub

In [0]:
!pip install vaderSentiment
!pip install emoji
!pip install textblob

You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-0fcb6c56-2406-4606-8219-b700d206538d/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-0fcb6c56-2406-4606-8219-b700d206538d/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 636 kB 11.1 MB/s 
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-0fcb6c56-2406-4606-8219-b700d206538d/bin/python -m pip install --upgrade pip' command.


In [0]:
#importacion de librerias necesarias para ejecutar el programa
import pyodbc
import pyspark
from pyspark.sql.functions import *
import pandas as pd
import emoji #Libreria para el manejo o interprentacion de emojis
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer #Libreria para analizar el sentimiento de comentario realizado en redes sociales
from textblob import TextBlob

In [0]:
#inicializacion de variables
data = []

In [0]:
def calificacion_textblob(texto):
    try:
        calificacion = TextBlob(texto)
        return calificacion.sentiment.polarity
    except:
        return 0

In [0]:
#cadena de conexión a la base de datos
try:
    conn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};Server=tcp:siigoanalytics.sql.azuresynapse.net,1433;Database=analyticsDWH;Uid=SiigoDWUsr;Pwd=r%4Z$pkaXz6k;Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;")
    
    #Se prepara la consulta para su ejecución y creación del Dataframe
    query = pd.read_sql_query("select id, score, templateAnswer from five9.five9Answers where templateAnswer not in ('')", conn)
    df = pd.DataFrame(query)
    
except Exception as e:
    print("Error: %s" % e)      
        

In [0]:
try:
    df['clean_templateAnswer'] = df['templateAnswer'].apply(lambda x : x.replace('\n',' ').strip())
    df.drop('templateAnswer', inplace=True, axis=1)
except Exception as e:
    print("Error: %s" % e)

In [0]:
analyzer = SentimentIntensityAnalyzer()
df['calif_compuesta'] = [analyzer.polarity_scores(x)['compound'] for x in df['clean_templateAnswer']]
df["des_calif_compuesta"] = df["calif_compuesta"].apply(lambda x: "Neutro" if x == 0 else("Positivo" if x > 0 else "Negativo"))

In [0]:
df['calif_TextBlod'] = df['clean_templateAnswer'].apply(lambda x: calificacion_textblob(x))
df["des_calif_TextBlod"] = df["calif_TextBlod"].apply(lambda x: "Neutro" if x == 0 else("Positivo" if x > 0 else "Negativo"))

In [0]:
df.head(100)

,id,score,clean_templateAnswer,calif_compuesta,des_calif_compuesta,calif_TextBlod,des_calif_TextBlod
0,530b6bb9-40c5-11ec-bbfd-005056a7303c,100,gracias,0.0,Neutro,0.0,Neutro
1,29af443d-3291-11ec-8cf7-00505ff4aad0,100,GRACIAS,0.0,Neutro,0.0,Neutro
2,bceb4741-32b7-11ec-ade4-00505fb70f54,100,si,0.0,Neutro,0.0,Neutro
3,e1b91b7d-3350-11ec-9b57-005056a70e34,100,Prueba Jhony,0.0,Neutro,0.0,Neutro
4,117e80f2-47e4-11ec-8c0f-005056ad2460,100,excelente persona,0.0,Neutro,0.0,Neutro
...,...,...,...,...,...,...,...
95,40a0a12f-3992-11ec-a123-005056ad2460,100,buen servicio,0.0,Neutro,0.0,Neutro
96,55339d76-3dbd-11ec-8604-005056ad8e2c,100,excelente atencion de hernan david 100%,0.0,Neutro,0.0,Neutro
97,ca56d3f0-4263-11ec-a564-005056a70c10,100,hola mi nombre es Gustavo jaramillo perez cc 7...,0.0,Neutro,0.0,Neutro
98,534fba60-3e47-11ec-8599-005056addbc1,100,EXCELENTE ATENCION,0.0,Neutro,0.0,Neutro


In [0]:
df['des_calif_TextBlod'].value_counts(normalize=True, ascending=False)*100

Out[70]: Neutro      93.712486
Positivo     5.900132
Negativo     0.387382
Name: des_calif_TextBlod, dtype: float64

In [0]:
df['des_calif_compuesta'].value_counts(normalize=True, ascending=False)*100

Out[71]: Neutro      74.305053
Negativo    15.427185
Positivo    10.267762
Name: des_calif_compuesta, dtype: float64

In [0]:
condicion = df['des_calif_compuesta'] == 'Negativo'

df[condicion]

,id,score,clean_templateAnswer,calif_compuesta,des_calif_compuesta,calif_TextBlod,des_calif_TextBlod
18,06eef0a5-3747-11ec-ab8b-00505fb70f53,100,LA ATENCION EXCELENTE DE LA ASESORA PERO NO ME...,-0.2960,Negativo,0.000,Neutro
28,01ffe283-3da4-11ec-9f32-005056a7de99,100,No me percaté que me habían escrito... disculpas,-0.2960,Negativo,0.000,Neutro
29,13744240-3da2-11ec-8fc7-00506009ccd5,50,No encontré lo requerido,-0.2960,Negativo,0.000,Neutro
37,00940a1d-3e3d-11ec-b833-005056ad527b,50,No estoy conforme con la respuesta de que si v...,-0.6249,Negativo,-0.375,Negativo
41,99664015-5e92-11ec-95c3-2bbf77dc5af7,0,No contestaron,-0.2960,Negativo,0.000,Neutro
...,...,...,...,...,...,...,...
23456,0182fe74-c29a-a098-a71f-a88e0c3c13de,0,"Contifico no liquida bien las importaciones, p...",-0.5267,Negativo,0.000,Neutro
23457,0183a9ec-d5b6-aa5d-997b-92b6f044bb3c,100,"Estimados buenos dias, por favor solicito su ...",-0.7579,Negativo,0.000,Neutro
23461,b8a509fc-e36d-11ec-f3ef-657378d0ed53,25,"Quería una sesoría en linea, no enviarme link ...",-0.2960,Negativo,0.000,Neutro
23477,01831a56-5665-a68a-b834-45aef877825e,0,no pudieron ayudarme con todo el requerimiento...,-0.2960,Negativo,0.000,Neutro
